In [ ]:
import requests
import html
import time
import pandas as pd
from bs4 import BeautifulSoup
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

The next function gets a URL and returns a beautiful soup object :


In [ ]:
def create_soup_from_url(url):
    selenium = webdriver.Chrome()
    selenium.get(url)    
    return BeautifulSoup(selenium.page_source, 'html.parser')

The function gets an ID of the page and its URL, extracts the relevant attributes, and returns a list with the data.

In [ ]:
def fetch_book_data(id, url):
    soup = create_soup_from_url(url)

    book_title = soup.find('h1', class_= 'Text Text__title1').get_text()

    book_author = soup.find('div', class_='ContributorLinksList').get_text()
    
    number_of_pages_raw = soup.find('div', class_='FeaturedDetails').find('p').get_text()
    number_of_pages_array = number_of_pages_raw.split(' ')
    numeric_number_of_pages = int(number_of_pages_array[0].replace(',', ""))

    counts_raw = soup.find('div', class_='RatingStatistics__meta')
    counts_array = counts_raw.find_all('span')
    number_of_ratings = int(counts_array[0].get_text().split("\xa0")[0].replace(',', ""))
    number_of_reviews = int(counts_array[1].get_text().split("\xa0")[0].replace(',', ""))

    book_rating = float(soup.find('div', class_='RatingStatistics__rating').get_text())
    
    publisher = soup.find('div', class_= 'TruncatedContent__text TruncatedContent__text--small').get_text()    
    
    book_data = [id, book_title, book_author,numeric_number_of_pages,number_of_reviews,number_of_ratings,book_rating]
    time.wait(3)
    return book_data

In this function, we approach the site, extract the data, and we use the sleep function between every request.

In [ ]:
def main():
    books_columns = ['id', 'book title','book_author','number of pages','number of reviews', 'number of ratings','publisher', 'book rating']
    goodreads_template = 'https://www.goodreads.com/book/show/{}'
    books_dataframe = pd.DataFrame(columns = books_columns)
    
    while books_dataframe.shape[0] < 7000:
        try:
            data = fetch_book_data(random.randint(1, 500000), goodreads_template.format(id))
            temp = {col: data[i] for i, col in enumerate(columns)}
            books_dataframe = books_dataframe.append(temp, ignore_index=True)
        except:
            continue

Extract the data Frame to a CSV file, to use later in the EDA 


In [ ]:
books_dataframe.to_csv('goodreads.csv')